Import Statements and Initializations

In [137]:
import os
import pathlib
import numpy as np
from skimage.util import view_as_windows
from skimage.io import imread
from skimage import io
import imageio
import torch
from torch.utils import data

WINDOW = (384, 384)
RAW_IMAGES_PATH = "/home/snamburi/Debarshi/raw_images/"
RAW_MASKS_PATH = "/home/snamburi/Debarshi/raw_masks/"
IMAGES_PATH = "/home/snamburi/Debarshi/data/images/"
MASKS_PATH = "/home/snamburi/Debarshi/data/masks/"

Creating big stack

In [91]:
def merge_stacks(path):
    stacks_list = os.listdir(path)
    big_stack = []
    for file_path in pathlib.Path(path).iterdir():
        stack = imageio.mimread(file_path)
        np_stack = np.asarray(stack)
        for slice_idx in range(len(np_stack)):
            patches = view_as_windows(np_stack[slice_idx], WINDOW, step=(64, 64))
            for row in range(patches.shape[0]):
                for col in range(patches.shape[1]):
                    big_stack.append(patches[row][col])
    return big_stack

In [92]:
image_stack = merge_stacks(RAW_IMAGES_PATH)
mask_stack = merge_stacks(RAW_MASKS_PATH)

In [93]:
print("Image patches info")
print("Total no of patches: {}".format(len(image_stack)))
print("Shape of each patch: {}".format(image_stack[0].shape))
print("Mask patches info")
print("Total no of patches: {}".format(len(mask_stack)))
print("Shape of each patch: {}".format(mask_stack[0].shape))
assert len(image_stack) == len(mask_stack)

Image patches info
Total no of patches: 2520
Shape of each patch: (384, 384)
Mask patches info
Total no of patches: 2520
Shape of each patch: (384, 384)


In [94]:
def calculate_threshold(image_stack):
    mean_dict = {}
    for slice_idx in range(len(image_stack)):
        mean_dict[slice_idx] = image_stack[slice_idx].mean()
    threshold = sum(mean_dict.values())/len(mean_dict.values())
    return threshold

In [95]:
threshold = calculate_threshold(image_stack)
threshold

29.361774655834914

In [109]:
def create_data(image_stack, mask_stack, threshold):
    for patch_idx in range(len(image_stack)):
        image = image_stack[patch_idx]
        mask = mask_stack[patch_idx]
        if(image.mean() >= threshold):
            io.imsave(f"{IMAGES_PATH}/{patch_idx}.tiff", image)
            io.imsave(f"{MASKS_PATH}/{patch_idx}.tiff", mask)

In [110]:
create_data(image_stack, mask_stack, threshold)

/tmp/ipykernel_31993/3388717805.py:7: UserWarning: /home/snamburi/Debarshi/data/masks/188.tiff is a low contrast image
  io.imsave(f"{MASKS_PATH}/{patch_idx}.tiff", mask)
/tmp/ipykernel_31993/3388717805.py:7: UserWarning: /home/snamburi/Debarshi/data/masks/189.tiff is a low contrast image
  io.imsave(f"{MASKS_PATH}/{patch_idx}.tiff", mask)
/tmp/ipykernel_31993/3388717805.py:7: UserWarning: /home/snamburi/Debarshi/data/masks/199.tiff is a low contrast image
  io.imsave(f"{MASKS_PATH}/{patch_idx}.tiff", mask)
/tmp/ipykernel_31993/3388717805.py:7: UserWarning: /home/snamburi/Debarshi/data/masks/200.tiff is a low contrast image
  io.imsave(f"{MASKS_PATH}/{patch_idx}.tiff", mask)
/tmp/ipykernel_31993/3388717805.py:7: UserWarning: /home/snamburi/Debarshi/data/masks/201.tiff is a low contrast image
  io.imsave(f"{MASKS_PATH}/{patch_idx}.tiff", mask)
/tmp/ipykernel_31993/3388717805.py:7: UserWarning: /home/snamburi/Debarshi/data/masks/202.tiff is a low contrast image
  io.imsave(f"{MASKS_PATH}

In [135]:
import cv2
maximum = 0
for file_path in pathlib.Path(MASKS_PATH).iterdir():
    seg = cv2.imread(str(file_path))
    mi, ma = np.min(seg), np.max(seg)
    n_classes = ma - mi + 1
    if n_classes > maximum:
        maximum = n_classes
print(maximum)
    # print(f"{str(file_path)} : {n_classes}")

3


In [143]:
import torch.nn.functional as F
F.one_hot(torch.arange(0, 5) % 3)

tensor([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 0],
        [0, 1, 0]])

In [145]:
sample = Image.open("data/masks/1000.tiff")
type(sample)b

PIL.TiffImagePlugin.TiffImageFile

In [147]:
a = np.zeros((4,3))


12